## Import Libraries

In [1]:
%matplotlib inline
from utils import *

from keras.layers import Activation,TimeDistributed
from numpy.random import choice

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Setup

In [2]:
path = '../../data/nietzsche/nietzsche.txt';
text = open(path).read().lower()
print('Corpus Length : ',len(text))

Corpus Length :  600893


In [3]:
!tail -n25 $path 

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [4]:
chars = sorted(list(set(text)));
vocab_size=len(chars)+1
print('Total unique chars : ',vocab_size)

Total unique chars :  58


In [5]:
chars.insert(0,'\0')

In [6]:
''.join(chars[:])

'\x00\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [7]:
char_indices={v:k for k,v in enumerate(chars)}
indices_char={k:v for k,v in enumerate(chars)}

In [8]:
idx=[char_indices[c] for c in text]

In [9]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [10]:
''.join([indices_char[a] for a in idx[:70]])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [53]:
maxlen=40
sentences = [];
next_chars = [];
for i in range(0,len(idx)-maxlen+1):
    sentences.append(idx[i:i+maxlen]);  
    next_chars.append(idx[i+1:i+maxlen+1]);

In [26]:
print('nb sequences : ',len(sentences))

nb sequences :  600854


In [54]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]],axis=0)

In [56]:
n_fac=24

In [57]:
model = Sequential([
    Embedding(vocab_size,n_fac,input_length=maxlen),
    LSTM(512,input_shape=(None,n_fac),return_sequences=True,dropout=0.2,recurrent_dropout=0.2,
        implementation=2),
    Dropout(0.2),
    LSTM(512,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,
        implementation=2),
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size)),
    Activation('softmax')
])

In [58]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 24)            1392      
_________________________________________________________________
lstm_3 (LSTM)                (None, 40, 512)           1099776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 40, 512)           2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 58)            29754     
_________________________________________________________________
activation_2 (Activation)    (None, 40, 58)            0         
Total para

In [59]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [60]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(1):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [ ]:
model.fit(sentences,np.expand_dims(next_chars,2),batch_size=64,epochs=1)

Epoch 1/1
 44352/600852 [=>............................] - ETA: 9513s - loss: 2.4389